In [74]:
import pandas as pd
import numpy as np
import datetime

# Подаем на вход инфомацию о закупках

In [8]:
df_supply = pd.read_csv('out_input_MS-b1-supply.csv')
df_supply.head()

,date,apple,pen
0,2006-01-01,35086,2730
1,2006-01-15,35002,2625
2,2006-02-01,34963,2759
3,2006-02-15,34869,2715
4,2006-03-01,35188,2599


## Подаем на вход инфомацию о продажах

In [9]:
df_sell = pd.read_csv('out_input_MS-b1-sell.csv')
df_sell.head()

,date,sku_num
0,2006-01-01,MS-b1-ap-48914c5b-14d2-4b20-bdaf-b2ff5d9f4f0c
1,2006-01-01,MS-b1-ap-6baf7287-3e6a-4728-a3b1-8613de51eef8
2,2006-01-01,MS-b1-ap-83d7b005-c7d9-4deb-93a2-a8f7606d02b5
3,2006-01-01,MS-b1-ap-9099c5de-028d-4b07-97be-6f6532b8674a
4,2006-01-01,MS-b1-ap-ee0ce843-a979-4798-a542-75a41d1610c3


## Подаем на вход информацию об инвентаре

In [29]:
df_inventory = pd.read_csv('out_input_MS-b1-inventory.csv')
df_inventory.head()

,date,apple,pen
0,2006-01-31,12157,811
1,2006-02-28,29859,2280
2,2006-03-31,42135,3317
3,2006-04-30,55235,4094
4,2006-05-31,66963,5158


## Исследование состояния склада на каждый день

In [30]:
total_apples = 0
total_pens = 0

In [62]:
state = df_sell.sku_num[0][:2]

In [75]:
res_df = pd.DataFrame(columns=['date', 'apple', 'pen'])

from_ind = 0
to_ind = 0
prev_date = '2006-01-01'

for cur_sell in df_sell.values:
    str_d = cur_sell[0]
    code = cur_sell[1][6:8]
    
    if prev_date != str_d:
        sup = df_supply[df_supply.date==prev_date]
        
        if sup.shape[0] == 1:
            total_apples += int(sup.apple)
            total_pens += int(sup.pen)

        cur_df = pd.DataFrame(columns=['date', 'apple', 'pen'], data=[[prev_date, total_apples, total_pens]])
        res_df = res_df.append(cur_df)
    
    prev_date = str_d

    if code == 'ap':
        total_apples -= 1
    else:
        total_pens -= 1


sup = df_supply[df_supply.date==str_d]
if sup.shape[0] == 1:
    total_apples += int(sup.apple)
    total_pens += int(sup.pen)
cur_df = pd.DataFrame(columns=['date', 'apple', 'pen'], data=[[str_d, total_apples, total_pens]])
res_df = res_df.append(cur_df)
res_df.head()


,date,apple,pen
0,2006-01-01,3173689,249530
0,2006-01-02,3171827,249387
0,2006-01-03,3169947,249216
0,2006-01-04,3168150,249063
0,2006-01-05,3166208,248930


## Исследование месячных данных о количестве сворованного товара 

In [64]:
total_stolen_apples = 0
total_stolen_pens = 0

In [65]:
stolen_df = pd.DataFrame(columns=['date', 'apple', 'pen'])

for x in df_inventory.values:
    cur_d = x[0]
    paper_apples = x[1]
    paper_pens = x[2]
    real_inv = res_df[res_df.date==x[0]]
    real_apples = int(real_inv.apple)
    real_pens = int(real_inv.pen)
    cur_apples = real_apples-paper_apples-total_stolen_apples
    cur_pens = real_pens-paper_pens-total_stolen_pens
    cur_df = pd.DataFrame(columns=['date', 'apple', 'pen'], data=[[cur_d, cur_apples, cur_pens]])
    total_stolen_apples += cur_apples
    total_stolen_pens += cur_pens
    stolen_df = stolen_df.append(cur_df)
stolen_df.head()

,date,apple,pen
0,2006-01-31,1570219,123489
0,2006-02-28,6,6
0,2006-03-31,7,6
0,2006-04-30,6,14
0,2006-05-31,8,1


## Исследование агрегированных данных об объемах продаж и количестве сворованной продукции по штату и году

In [85]:
cols = ['year', 'state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen']
year_df = pd.DataFrame(columns=cols)

cur_year = 2006
supply_pens = 0
supply_apples = 0
ind = 0

for x in df_supply.values:
    cur_date = x[0]
    supply_apples += x[1]
    supply_pens += x[2]
    
    if ind % 24 == 23:
        cur_date = cur_date[:-2]+'31'
        apple_sold = supply_apples-int(res_df[res_df.date==cur_date].apple)
        pen_sold = supply_pens-int(res_df[res_df.date==cur_date].pen)
        pen_stolen = int(res_df[res_df.date==cur_date].pen)-int(df_inventory[df_inventory.date==cur_date].pen)
        apple_stolen = int(res_df[res_df.date==cur_date].apple)-int(df_inventory[df_inventory.date==cur_date].apple)
        cur_df = pd.DataFrame(columns=cols, data=[[cur_year, state, apple_sold, apple_stolen, pen_sold, pen_stolen]])
        year_df = year_df.append(cur_df)
        cur_year += 1
    ind += 1
year_df.head()

,year,state,apple_sold,apple_stolen,pen_sold,pen_stolen
0,2006,MS,-2459077,3140496,-194401,247047
0,2007,MS,-1777796,3140564,-142462,247123
0,2008,MS,-1092686,3140642,-90430,247196
0,2009,MS,-410390,3140723,-38019,247292
0,2010,MS,270021,3140791,14581,247388
